# SVI Calibration Demo (Equity)

This notebook demonstrates SVI volatility surface calibration for equity options.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from derivx.vol.svi import fit_svi_strike_slice, svi_iv, basic_no_arb_checks

%matplotlib inline


## Generate synthetic smile data


In [ ]:
# True SVI parameters
true_params = {"a": 0.02, "b": 0.3, "rho": -0.2, "m": 0.0, "s": 0.4}
F = 100.0
Ks = np.linspace(60, 140, 20)

# Generate IVs from true params + noise
ivs_true = np.array([svi_iv(K, true_params, F) for K in Ks])
np.random.seed(42)
ivs_obs = ivs_true + 0.001 * np.random.randn(len(Ks))

print(f"Generated {len(Ks)} strikes from {Ks[0]:.0f} to {Ks[-1]:.0f}")


## Calibrate SVI and plot results


In [ ]:
params_est = fit_svi_strike_slice(Ks, ivs_obs, F)
ivs_fit = np.array([svi_iv(K, params_est, F) for K in Ks])
rmse = np.sqrt(np.mean((ivs_fit - ivs_true) ** 2))

print("Estimated params:", {k: f"{v:.4f}" for k, v in params_est.items()})
print(f"RMSE: {rmse:.6f}")

# Plot
plt.figure(figsize=(10, 6))
plt.scatter(Ks, ivs_obs, label='Observed IVs', alpha=0.6, s=50)
plt.plot(Ks, ivs_fit, label='SVI fit', linewidth=2, color='red')
plt.plot(Ks, ivs_true, label='True smile', linewidth=1, linestyle='--', alpha=0.7, color='green')
plt.xlabel('Strike')
plt.ylabel('Implied Volatility')
plt.title('SVI Smile Calibration (Equity)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# No-arb diagnostics
diag = basic_no_arb_checks([params_est])
print("No-arbitrage diagnostics:", diag)
